# Chocolate Sales Challenge

[Kaggle](https://www.kaggle.com/datasets/saidaminsaidaxmadov/chocolate-sales)

## Импорт библиотек

In [259]:
import pathlib
import pickle

import kagglehub
import pandas as pd
import sklearn.compose
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.linear_model
import sklearn.ensemble
import sklearn.metrics
import sklearn.calibration

## Просмотр данных

In [260]:
path_to_directory = pathlib.Path(
    kagglehub.dataset_download('saidaminsaidaxmadov/chocolate-sales'))

path_to_data = path_to_directory / 'Chocolate Sales (2).csv'
print(f'{path_to_directory = }')
print(f'{path_to_data = }')

Using Colab cache for faster access to the 'chocolate-sales' dataset.
path_to_directory = PosixPath('/kaggle/input/chocolate-sales')
path_to_data = PosixPath('/kaggle/input/chocolate-sales/Chocolate Sales (2).csv')


In [261]:
df = pd.read_csv(path_to_data, parse_dates=['Date'])
df.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,"$5,320.00",180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,"$7,896.00",94
2,Gigi Bohling,India,Peanut Butter Cubes,07/07/2022,"$4,501.00",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27/04/2022,"$12,726.00",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24/02/2022,"$13,685.00",184


In [262]:
df.tail()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
3277,Karlen McCaffrey,Australia,Spicy Special Slims,17/05/2024,"$5,303.58",354
3278,Jehu Rudeforth,USA,White Choc,07/06/2024,"$7,339.32",121
3279,Ches Bonnell,Canada,Organic Choco Syrup,26/07/2024,$616.09,238
3280,Dotty Strutley,India,Eclairs,28/07/2024,"$2,504.62",397
3281,Karlen McCaffrey,India,70% Dark Bites,23/05/2024,"$5,915.87",355


In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sales Person   3282 non-null   object
 1   Country        3282 non-null   object
 2   Product        3282 non-null   object
 3   Date           3282 non-null   object
 4   Amount         3282 non-null   object
 5   Boxes Shipped  3282 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 154.0+ KB


In [264]:
df.describe()

,Boxes Shipped
count,3282.000000
mean,164.666971
std,124.024736
min,1.000000
25%,71.000000
50%,137.000000
75%,232.000000
max,778.000000


In [265]:
df.describe(include=['object'])

,Sales Person,Country,Product,Date,Amount
count,3282,3282,3282,3282,3282
unique,25,6,22,504,3013
top,Kelci Walkden,Australia,Eclairs,13/01/2024,"$2,317.00"
freq,162,615,180,16,5


In [266]:
str_features = df.describe(include=['object']).columns
str_features

Index(['Sales Person', 'Country', 'Product', 'Date', 'Amount'], dtype='object')

In [267]:
for feature in str_features:
  print(f'{feature}:', ', '.join(pd.unique(df[feature])))

Sales Person: Jehu Rudeforth, Van Tuxwell, Gigi Bohling, Jan Morforth, Oby Sorrel, Gunar Cockshoot, Brien Boise, Rafaelita Blaksland, Barr Faughny, Mallorie Waber, Karlen McCaffrey, Marney O'Breen, Beverie Moffet, Roddy Speechley, Curtice Advani, Husein Augar, Kaine Padly, Dennison Crosswaite, Wilone O'Kielt, Andria Kimpton, Kelci Walkden, Camilla Castle, Madelene Upcott, Dotty Strutley, Ches Bonnell
Country: UK, India, Australia, New Zealand, USA, Canada
Product: Mint Chip Choco, 85% Dark Bars, Peanut Butter Cubes, Smooth Sliky Salty, 99% Dark & Pure, After Nines, 50% Dark Bites, Orange Choco, Eclairs, Drinking Coco, Organic Choco Syrup, Milk Bars, Spicy Special Slims, Fruit & Nut Bars, White Choc, Manuka Honey Choco, Almond Choco, Raspberry Choco, Choco Coated Almonds, Baker's Choco Chips, Caramel Stuffed Bars, 70% Dark Bites
Date: 04/01/2022, 01/08/2022, 07/07/2022, 27/04/2022, 24/02/2022, 06/06/2022, 25/01/2022, 24/03/2022, 20/04/2022, 04/07/2022, 13/01/2022, 10/03/2022, 28/07/2022

## Обработка данных

In [268]:
df['Amount'] = df['Amount'].replace({'\$': '', ',': ''}, regex=True)
df['Amount'] = pd.to_numeric(df['Amount'])
df['Amount'].head()

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1438467415.py:1: SyntaxWarning: invalid escape sequence '\$'
  df['Amount'] = df['Amount'].replace({'\$': '', ',': ''}, regex=True)


,Amount
0,5320.0
1,7896.0
2,4501.0
3,12726.0
4,13685.0


In [269]:
df.describe()

,Amount,Boxes Shipped
count,3282.000000,3282.000000
mean,6030.338775,164.666971
std,4393.980200,124.024736
min,7.000000,1.000000
25%,2521.495000,71.000000
50%,5225.500000,137.000000
75%,8556.842500,232.000000
max,26170.950000,778.000000


In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sales Person   3282 non-null   object 
 1   Country        3282 non-null   object 
 2   Product        3282 non-null   object 
 3   Date           3282 non-null   object 
 4   Amount         3282 non-null   float64
 5   Boxes Shipped  3282 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 154.0+ KB


In [271]:
if 'Date' in df:
  df = df.drop('Date', axis=1).drop_duplicates()

In [272]:
df.columns

Index(['Sales Person', 'Country', 'Product', 'Amount', 'Boxes Shipped'], dtype='object')

In [273]:
df.describe(include=['object'])

,Sales Person,Country,Product
count,3282,3282,3282
unique,25,6,22
top,Kelci Walkden,Australia,Eclairs
freq,162,615,180


In [274]:
pd.get_dummies(df.copy()).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Amount                            3282 non-null   float64
 1   Boxes Shipped                     3282 non-null   int64  
 2   Sales Person_Andria Kimpton       3282 non-null   bool   
 3   Sales Person_Barr Faughny         3282 non-null   bool   
 4   Sales Person_Beverie Moffet       3282 non-null   bool   
 5   Sales Person_Brien Boise          3282 non-null   bool   
 6   Sales Person_Camilla Castle       3282 non-null   bool   
 7   Sales Person_Ches Bonnell         3282 non-null   bool   
 8   Sales Person_Curtice Advani       3282 non-null   bool   
 9   Sales Person_Dennison Crosswaite  3282 non-null   bool   
 10  Sales Person_Dotty Strutley       3282 non-null   bool   
 11  Sales Person_Gigi Bohling         3282 non-null   bool   
 12  Sales 

## Разделение данных

In [275]:
X, y = df.drop('Amount', axis=1), df['Amount']

In [276]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
  X,
  y,
  test_size=0.33,
  random_state=42,
)

In [277]:
len(X_train)

2198

## Определеине обработки данных

In [278]:
numerical_features = X_train.select_dtypes(include='number').columns
categorical_features = X_train.select_dtypes(include='object').columns
print(
    f'{numerical_features = }\n{categorical_features = }'
)

numerical_features = Index(['Boxes Shipped'], dtype='object')
categorical_features = Index(['Sales Person', 'Country', 'Product'], dtype='object')


In [279]:
preprocessor = sklearn.compose.ColumnTransformer(
  transformers=[
      ('num', sklearn.preprocessing.MinMaxScaler(), numerical_features),
      ('cat', sklearn.preprocessing.OneHotEncoder(
          sparse_output=False,
          handle_unknown='ignore',
      ), categorical_features)
])

## Обучение моделей

In [280]:
model = sklearn.ensemble.RandomForestRegressor()
pipeline = sklearn.pipeline.Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', model)
])

In [281]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Boxes Shipped'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['Sales Person', 'Country', 'Product'], dtype='object'))])),
                ('regressor', RandomForestRegressor())])

## Оценка качества

In [282]:
rmse = sklearn.metrics.root_mean_squared_error(
    y_test,
    pipeline.predict(X_test),
)
print(f'{rmse = :.2f} $')

rmse = 2511.75 $


In [283]:
x, y = df.head(1).drop('Amount', axis=1), df.head(1)['Amount'].item()
print(
    f'True pred: {y:.2f} $\n',
    f'Model pred: {pipeline.predict(x)[0]:.2f} $',
)

True pred: 5320.00 $
 Model pred: 6252.49 $


## Сохранение Pipeline'а

In [284]:
with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [285]:
with open('pipeline.pkl', 'rb') as file:
    pipline = pickle.load(file)

In [286]:
x, y = df.head(1).drop('Amount', axis=1), df.head(1)['Amount'].item()
print(
    f'True pred: {y:.2f} $\n',
    f'Model pred: {pipeline.predict(x)[0]:.2f} $',
)

True pred: 5320.00 $
 Model pred: 6252.49 $
